In [1]:
import re
import csv
import json
import twint
import networkx as nx
from networkx.readwrite import json_graph
import requests
import os.path
from os import listdir
from os.path import isfile, join
import pandas as pd 

In [2]:
def read_json_graph(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

In [3]:
# twint does not provide the support to get the usernames of the retweeters yet. Hence the function
def get_retweeters_list(username, tweet_id):
    # get the data of retweets
    r = requests.get('https://twitter.com/i/activity/retweeted_popup?id='+tweet_id)
    # use the grep in order to getch the retweeters
    text = r.text
    x = re.findall('div class=\\\\"account  js-actionable-user js-profile-popup-actionable \\\\" data-screen-name=\\\\"(.+?)\\\\" data-user-id=\\\\"', text)
    return x

In [4]:
# get the tweets that have been retweeted
def get_tweet_id(username):
    # open the tweets file of the user
    retweeted_tweets = []
    ids_list = []
    try:
        with open('StronglyConnectedUsers/'+username+'.csv') as csvfile:
            readCSV = csv.reader(csvfile, delimiter=',')
            for row in readCSV:
                if row[3] != '0' and row[1] != 'id' and row[2] == username :
                    retweeted_tweets.append(row[1])
    except:
        pass
    # return a list of tweet ID's
    return(retweeted_tweets)

In [5]:
data = pd.read_csv("/Users/savanpatel/Desktop/sfsu/research/spamDetection/SDUTDPRev3/DataProcessing/DominantTopics.csv") 
data.head()

,Unnamed: 0,Document_No,First_Topic,Second_Topic,Third_Topic,Document_ID
0,0,0,3.0,1.0,5.0,clean_deborah_fraser_.csv
1,1,1,7.0,5.0,4.0,clean_naolifant.csv
2,2,2,7.0,5.0,2.0,clean_hotbwoyp.csv
3,3,3,4.0,3.0,5.0,clean_Tnngcobo.csv
4,4,4,1.0,4.0,5.0,clean_mshalisto.csv


In [6]:
max_topics = data['First_Topic'].max() + 1
print(max_topics)

8.0


In [7]:
def interested_topic(username):
    row = data[data['Document_ID'].str.contains(username)]
    return row['First_Topic']

In [8]:
interested_topic('RedneckTWD')

Series([], Name: First_Topic, dtype: float64)

In [9]:
sub_graphs = ['json_graph.json']
print(len(sub_graphs))
complete_list_of_users = []
node_dict = {}
for item1 in sub_graphs:
    print(item1)
    # for each subgraph, get all the nodes and extract their top tweets
    G = nx.Graph()
    G = read_json_graph(item1)
    list_of_user = list(G.nodes)
    node_dict[item1] = list_of_user
    for item in list_of_user:
        complete_list_of_users.append(item)
print(node_dict)
print(len(complete_list_of_users))
with open('complete_list_of_users.txt', 'w') as f:  
    for user in complete_list_of_users:
        f.write('%s\n' % user)

1
json_graph.json
{'json_graph.json': ['WhiteMediafrica', 'Uzalo_SABC1', 'oscar_Udeddie', 'Beevatar', 'DumiB_Nabalalel', 'khanyacomms', 'themusicimbizo', 'nelisiwe_sibiya', 'OmegaXDreams', 'NeoNtlatleng', 'Cafeskyzer11', 'SowetoTVchannel', 'ProtectionIngwe', 'soozandbnnyt', 'sa_artists', 'nam_radio', 'duduWagaG', 'Simps_majola', 'PASTABDXLL', 'durbanjozinites', 'ZakweSA', 'Alyscia_C', 'LalombaMusic', 'mlutheartist', 'SADJAwards', 'Ladylatta', 'LwaziGB_SA', 'jet_dynamics', 'MillyMashile', 'TheMusicDA', 'MusicBoxEntSA', 'RealDjTerance', 'MuNgcolosi', 'MajorArtOffice', 'Thokozani_Hlong', 'blose2sizwe', 'TongaatBeach', 'IamParadise_Sa', 'Tebogo_Louw', 'sanele_mkhizeh', 'JarredLove13', 'DiseboMusic', 'BoogieHarrySA', 'NocxyMabika', 'BishopthandaM', 'Thabzero_NUZ', 'hotbwoyp', 'Sjerere_dj', 'sosha745', 'nathi_mankayi', 'Zuka_DC', 'MeetKingKoolie', 'KaMavusoSA', 'Iambusin_sa', 'IAMYNGKING', 'MlixxFR', 'Tnngcobo', 'Oz_Lindokuhle', 'siphelele_fuze', 'Karabo_Mokgoko', 'PoetMastermind', 'seanhcho

In [10]:
'''
After saving the users to acomplete_list_of_users.txt, run the tweet collection code from Pycharm.
Due to the underlying issues of jupyter, data collection code breaks while executing it here.
'''

'\nAfter saving the users to acomplete_list_of_users.txt, run the tweet collection code from Pycharm.\nDue to the underlying issues of jupyter, data collection code breaks while executing it here.\n'

In [11]:
# delete the old file and create a new blank one
if os.path.isfile("vector.csv"):
    print("file exists")
    os.remove("vector.csv")
open("vector.csv", 'a').close()

In [12]:
with open('vector.csv', 'r+', newline='') as csvfile :
    vector = csv.writer(csvfile, delimiter=',')
    heading = list(range(1, int(max_topics)+1))
    vector.writerow(['tweet_id']+heading)
    buckets = [0] * int(max_topics)
    for user in complete_list_of_users:
        # get list of tweet_id of retweeted tweets
        ids_of_retweeted_tweets = get_tweet_id(user)
        for id_tweet in ids_of_retweeted_tweets:
            # add a row to the csv file with the tweet_id
            row = [id_tweet]
            row.extend((buckets))
            # get the usernames of the retweeters
            retweeters = []
            retweeters = get_retweeters_list(user, id_tweet)
            # add tweet's oroginal poster to the list of retweeters to the vector
            retweeters.append(user)
            #print(id_tweet)
            #print(retweeters)
            # for each retweeter check if the retweeter is in the complete list of users
            for retweeter in retweeters:
                if retweeter in complete_list_of_users:
                    # if the retweeter exists in the complete list of users check which topic he/she is interested in
                    try:
                        topic = int(interested_topic(retweeter)) + 1
                    except TypeError:
                        pass
                    #print(retweeter+"is interested in "+str(topic))
                    # get the existing count for that topic
                    count = row[topic]
                    count = count + 1
                    row[topic] = count
            vector.writerow(row)
                            

In [13]:
'''
Extracting tweets with more than 1 value in the vector or having a single value of more than 1
'''

'\nExtracting tweets with more than 1 value in the vector or having a single value of more than 1\n'

In [14]:
# delete the old file and create a new blank one
if os.path.isfile("reducedVector.csv"):
    print("file exists")
    os.remove("reducedVector.csv")
open("reducedVector.csv", 'a').close()

In [15]:
totalVectors = 0
oneNonZero = 0
OneNonZero = 0
TwoNonZero = 0
ThreeNonZero = 0
FourNonZero = 0
moreThanFourNonZero = 0

In [16]:
with open('vector.csv', 'r', newline='') as csvfile :
    vectors = csv.reader(csvfile, delimiter=',')
    for vector in vectors:
        if vector[0] == 'tweet_id':
            continue
        totalVectors = totalVectors + 1
        nonZeroValues = 0
        for item in range(1,len(vector)):
            if int(vector[item]) > 0:
                nonZeroValues = nonZeroValues + 1
        if nonZeroValues > 1:
            with open('reducedVector.csv', 'a', newline='') as reducedFile:
                vector2 = csv.writer(reducedFile, delimiter=',')
                vector2.writerow(vector)
        if nonZeroValues >= 3:
            with open('ThreeNonZeroVector.csv', 'a', newline='') as reducedFile:
                vector2 = csv.writer(reducedFile, delimiter=',')
                vector2.writerow(vector)